### setting up the environment the first time

Complications always arise when setting this up for the first time. So I'm going to *attempt* to standardize this somewhat for me.

Current config: Windows10pro but running on WSL2

## Need to install an X11 graphical monitor on windows. 
Interents recommented [VcXsrv](https://sourceforge.net/projects/vcxsrv/) [Video Instruction](https://www.youtube.com/watch?v=4SZXbl9KVsw) [full writeup](https://www.gregbrisebois.com/posts/chromedriver-in-wsl2/)

Also have to update firewall permission to allow VcXsrv

.
## to install the google chrome webdiver ##

Step 1 – Add Google Chrome PPA

First, add (if not added already) the Google Chrome repository on your system using the following command. While using PPA to our system we also receive the latest updates whenever you check for system updates.

<code> wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add - </code>

Next, create a PPA file for Google chrome on your system by running command:

<code> sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list' </code>

Step 2 – Install or Upgrade Google Chrome

After adding the Google Chrome repository in our system use following commands to install the latest Google Chrome stable release. If you already have installed an older version, It will upgrade the currently installed version with the recent stable version.

<code> sudo apt-get update
sudo apt-get install google-chrome-stable </code>

In [11]:
# Data Basics
import pandas as pd
import numpy as np
from datetime import datetime as dt
pd.options.display.float_format = "{:.0f}".format

# scraping
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# to install ChromeDriverManager() on first use
# chrome_options = Options()
# chrome_options.add_argument("--no-sandbox")
# chrome_options.addArguments("--no-sandbox");
# chrome_options.add_arguments("--disable-dev-shm-usage")
# webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
# webdriver.Chrome(ChromeDriverManager().install())

In [17]:
class SiteReader():
    def __init__(self):
        """ Headless options for built script """
        chrome_options = Options()
        # Comment out when developing / debugging
        chrome_options.add_argument("--no-sandbox")
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=800,600")
        chrome_options.add_argument("--remote-debugging-port=9222") 
        self.driver = webdriver.Chrome(options=chrome_options)

        # Prototyping
        self.driver = webdriver.Chrome()

In [18]:
bot = SiteReader()

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (chrome not reachable)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


NameError: name 'ChromeDriverManager' is not defined

In [21]:
last_page = False
i = 1
ulist = []
while last_page == False and i < 10:
    print(i)
    ulist.append(i)
    i += 1

1
2
3
4
5
6
7
8
9


In [22]:
to_save = pd.DataFrame(ulist)

In [23]:
to_save.to_csv('delete_me.csv')

In [26]:
print(i,ulist[-1])

10 9
